In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/My Drive/dacon

/content/drive/My Drive/dacon


## 1. 데이터 불러오기

In [3]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
#!cp "/content/drive/MyDrive/월간데이콘 12/data/open/data_2.zip" "data_2.zip"
# data_2.zip을 현재 디렉터리에 압축해제
# !unzip "data.zip"

In [4]:
# from google.colab import output
# # 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
# #dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# # 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
# #test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# # 출력 결과 지우기
output.clear()

In [5]:
!mkdir "./models"

In [6]:
ls

 10.pth                      'EfficientNet_0.927(1).pth'
 11.pth                      'EfficientNet_0.927(2).csv'
 12.pth                      'EfficientNet_0.927(2).pth'
 13.pth                       EfficientNet_0.927.csv
 14.pth                       EfficientNet_0.927.pth
 data_2.zip                   EfficientNet_0.928.csv
 dirty_mnist/                 EfficientNet_0.928.pth
 dirty_mnist_2nd_answer.csv   mnist_data.zip
 dirty_mnist_2nd.zip          models/
 EfficientNet_0.926.csv       sample_submission.csv
 EfficientNet_0.926.pth       test_dirty_mnist/
'EfficientNet_0.927(1).csv'   test_dirty_mnist_2nd.zip


## 2. Library Import

In [7]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=851984d6897990bfb9d2d0349222b19d1641beb1140cadb2a41a0d128e0eb655
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from google.colab import output
from efficientnet_pytorch import EfficientNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정

## 3. Dataset 구성

In [9]:
from typing import Tuple, Sequence, Callable
from torch import nn, Tensor
dirty_mnist_answer = pd.read_csv("dirty_mnist_2nd_answer.csv")
# dirty_mnist라는 디렉터리 속에 들어있는 파일들의 이름을 
# namelist라는 변수에 저장
namelist = os.listdir('./dirty_mnist/')

# unmpy를 tensor로 변환하는 ToTensor 정의
class ToTensor(object):
    """numpy array를 tensor(torch)로 변환합니다."""
    def __call__(self, sample):
        label = sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        #image = image.transpose((2, 0, 1))
        return torch.FloatTensor(label)
# to_tensor 선언
to_tensor = T.Compose([
                        # T.RandomHorizontalFlip(p=0.5),
                        # T.RandomVerticalFlip(p=0.5),
                        ToTensor(),
                        #T.Normalize((0.1307,),(0.3081,))
                         
                        
                        
                    ])
trans_train=T.Compose(
    [
     T.RandomAffine(180),
     T.ToTensor(),
     T.GaussianBlur((3,3)),
     T.Normalize((0.1307,),(0.3081,))
     
    ]
)

trans_test=T.Compose(
    [
     T.ToTensor(),
     T.GaussianBlur((3,3)),
     T.Normalize((0.1307,),(0.3081,))
    ]
)

class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms=None,#미리 선언한 to_tensor를 transforms로 받음
                 augmentations=None):
        
        self.dir_path = dir_path # 데이터의 이미지가 저장된 디렉터리 경로
        self.meta_df = meta_df # 데이터의 인덱스와 정답지가 들어있는 DataFrame

        self.transforms = transforms# Transform
        self.augmentations = augmentations # Augmentation
        
    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, index):
        # 폴더 경로 + 이미지 이름 + .png => 파일의 경로
        # 참고) "12".zfill(5) => 000012
        #       "146".zfill(5) => 000145
        # cv2.IMREAD_GRAYSCALE : png파일을 채널이 1개인 GRAYSCALE로 읽음
        
        image = cv2.imread(self.dir_path +\
                          str(self.meta_df.iloc[index,0]).zfill(5) + '.png',
                          cv2.IMREAD_GRAYSCALE)
        
        # 0 ~ 255의 값을 갖고 크기가 (256,256)인 numpy array를
        # 0 ~ 1 사이의 실수를 갖고 크기가 (256,256,1)인 numpy array로 변환
        
        # image = Image.fromarray(image)
        # 정답 numpy array생성(존재하면 1 없으면 0)
        label = self.meta_df.iloc[index, 1:].values.astype('float')
        label=torch.tensor(label)
        #image = (image).astype('float')[..., np.newaxis]
        image = Image.fromarray(image)
        
        image=trans_train(image)
        
        #label=self.transforms(label)
        sample = {'image': image, 'label': label}

        # transform 적용
        # numpy to tensor
        # if self.transforms:
        #     sample = self.transforms(sample)
        

        return sample

class DatasetMNIST_test(torch.utils.data.Dataset):
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms=None,#미리 선언한 to_tensor를 transforms로 받음
                 augmentations=None):
        
        self.dir_path = dir_path # 데이터의 이미지가 저장된 디렉터리 경로
        self.meta_df = meta_df # 데이터의 인덱스와 정답지가 들어있는 DataFrame

        self.transforms = transforms# Transform
        self.augmentations = augmentations # Augmentation
        
    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, index):
        # 폴더 경로 + 이미지 이름 + .png => 파일의 경로
        # 참고) "12".zfill(5) => 000012
        #       "146".zfill(5) => 000145
        # cv2.IMREAD_GRAYSCALE : png파일을 채널이 1개인 GRAYSCALE로 읽음
        
        image = cv2.imread(self.dir_path +\
                          str(self.meta_df.iloc[index,0]).zfill(5) + '.png',
                          cv2.IMREAD_GRAYSCALE)
        
        # 0 ~ 255의 값을 갖고 크기가 (256,256)인 numpy array를
        # 0 ~ 1 사이의 실수를 갖고 크기가 (256,256,1)인 numpy array로 변환
        
        # image = Image.fromarray(image)
        # 정답 numpy array생성(존재하면 1 없으면 0)
        label = self.meta_df.iloc[index, 1:].values.astype('float')
        label=torch.tensor(label)
        #image = (image).astype('float')[..., np.newaxis]
        image = Image.fromarray(image)
        
        image=trans_test(image)
        
        #label=self.transforms(label)
        sample = {'image': image, 'label': label}

        # transform 적용
        # numpy to tensor
        # if self.transforms:
        #     sample = self.transforms(sample)
        

        return sample

## 4. 학습 모델 구성

In [ ]:
# nn.Module을 상속 받아 MultiLabelResnet를 정의
class MultiLabelResnet(nn.Module):
    def __init__(self):
        super(MultiLabelResnet, self).__init__()
        self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        # self.resnet = models.densenet121(pretrained=True)
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b7')
        self.FC = nn.Linear(1000,26)
    

    def forward(self, x):
        # resnet의 입력은 [3, N, N]으로
        # 3개의 채널을 갖기 때문에
        # resnet 입력 전에 conv2d를 한 층 추가
        x = F.relu(self.conv2d(x))
        x = F.relu(self.efficientnet(x))

        # resnet18을 추가
        x = torch.sigmoid(self.FC(x))

        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid를 적용
        #x = torch.sigmoid(self.FC(x))
        return x
# 모델 선언
model = MultiLabelResnet()
model

## 5. 학습

In [ ]:
# cross validation을 적용하기 위해 KFold 생성
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

#print(train_test_split(dirty_mnist_answer, test_size=0.3 , random_state=123))

# dirty_mnist_answer에서 train_idx와 val_idx를 생성
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(dirty_mnist_answer),1):
#for (trn_idx, val_idx) in enumerate(train_test_split(dirty_mnist_answer, test_size=0.3 , random_state=123)): 
    #print(f'[fold: {fold_index}]')
    # cuda cache 초기화
    torch.cuda.empty_cache()
    #print(val_idx)
    #train fold, validation fold 분할
    train_answer = dirty_mnist_answer.iloc[trn_idx]
    test_answer  = dirty_mnist_answer.iloc[val_idx]
    #train_answer=trn_idx
    #test_answer=val_idx
    #print(test_answer)
    #print(train_answer)
    #Dataset 정의
    train_dataset = DatasetMNIST("dirty_mnist/", train_answer)
    valid_dataset = DatasetMNIST_test("dirty_mnist/", test_answer)

    

    #DataLoader 정의
    train_data_loader = DataLoader(
        train_dataset,
        batch_size = 16,
        shuffle = True,
        num_workers = 3
    )
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size = 16,
        shuffle = True,
        num_workers = 3
    )
  

    # 모델 선언
    # model = MultiLabelResnet()
    # model.to(device)# gpu에 모델 할당
    model = torch.load('./EfficientNet_0.928.pth')
    model.to(device)

    # 훈련 옵션 설정
    optimizer = torch.optim.Adam(model.parameters(),
                                lr = 0.0001,
                                 weight_decay=0
                                 )
    # 0 <= epoch < 5 -> 0.0005, 5 <= epoch < 10 -> 0.0005 * 0.75 ...
    # lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
    #                                              step_size = 5,
    #                                              gamma = 0.9)

    lr_scheduler= torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    criterion = torch.nn.MSELoss()
    
    # 훈련 시작
    valid_acc_max = 0
    for epoch in range(20):
        # 1개 epoch 훈련
        train_acc_list = []
        with tqdm(train_data_loader,#train_data_loader를 iterative하게 반환
                total=train_data_loader.__len__(), # train_data_loader의 크기
                unit="batch") as train_bar:# 한번 반환하는 smaple의 단위는 "batch"
            for sample in train_bar:
                train_bar.set_description(f"Train Epoch {epoch}")
                # 갱신할 변수들에 대한 모든 변화도를 0으로 초기화
                # 참고)https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                # tensor를 gpu에 올리기 
                images = images.to(device)
                labels = labels.to(device)

                # 모델의 dropoupt, batchnormalization를 train 모드로 설정
                model.train()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.set_grad_enabled(True):
                    # 모델 예측
                    probs  = model(images).double()
                    # loss 계산
                    loss = criterion(probs, labels)
                    # 중간 노드의 gradient로
                    # backpropagation을 적용하여
                    # gradient 계산
                    loss.backward()
                    # weight 갱신
                    optimizer.step()

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    train_acc_list.append(batch_acc)
                    train_acc = np.mean(train_acc_list)

                # 현재 progress bar에 현재 미니배치의 loss 결과 출력
                train_bar.set_postfix(train_loss= loss.item(),
                                      train_acc = train_acc)
                

        # 1개 epoch학습 후 Validation 점수 계산
        valid_acc_list = []
        with tqdm(valid_data_loader,
                total=valid_data_loader.__len__(),
                unit="batch") as valid_bar:
            for sample in valid_bar:
                valid_bar.set_description(f"Valid Epoch {epoch}")
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)

                # 모델의 dropoupt, batchnormalization를 eval모드로 설정
                model.eval()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.no_grad():
                    # validation loss만을 계산
                    probs  = model(images).double()
                    valid_loss = criterion(probs, labels)

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    valid_acc_list.append(batch_acc)

                valid_acc = np.mean(valid_acc_list)
                valid_bar.set_postfix(valid_loss = valid_loss.item(),
                                      valid_acc = valid_acc)
            
        # Learning rate 조절
        lr_scheduler.step()

        # 모델 저장
        if valid_acc_max < valid_acc:
            valid_acc_max = valid_acc
            best_model = model
            MODEL = "EfficientNet"
            # 모델을 저장할 구글 드라이브 경로
            path = "/content/drive/MyDrive/dacon/models/"
            torch.save(best_model, f'{path}{fold_index}_{MODEL}_{valid_loss.item():2.4f}_epoch_{epoch}.pth')

    # 폴드별로 가장 좋은 모델 저장
    best_models.append(best_model)

Train Epoch 4:  63%|██████▎   | 1576/2500 [18:20<10:46,  1.43batch/s, train_acc=0.923, train_loss=0.0426]

## 6. 학습 결과 확인

## 7. 앙상블 적용

In [ ]:
model = torch.load('./EfficientNet_0.928.pth')
model.to(device)

In [ ]:
#test Dataset 정의
sample_submission = pd.read_csv("sample_submission.csv")
test_dataset = DatasetMNIST_test("test_dirty_mnist/", sample_submission)
batch_size = 16
test_data_loader = DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 3,
    drop_last = False
)

In [ ]:
predictions_list = []
# 배치 단위로 추론
prediction_df = pd.read_csv("sample_submission.csv")

# 5개의 fold마다 가장 좋은 모델을 이용하여 예측
# print(best_model)
# 0으로 채워진 array 생성
prediction_array = np.zeros([prediction_df.shape[0],
                              prediction_df.shape[1] -1])
for idx, sample in enumerate(test_data_loader):
    with torch.no_grad():
        # 추론
        model.eval()
        images = sample['image']
        images = images.to(device)
        probs  = model(images)
        probs = probs.cpu().detach().numpy()
        preds = probs

        # 예측 결과를 
        # prediction_array에 입력
        batch_index = batch_size * idx
        prediction_array[batch_index: batch_index + images.shape[0],:]\
                      = preds.astype(float)
                      
# 채널을 하나 추가하여 list에 append
predictions_list.append(prediction_array[...,np.newaxis])

In [ ]:
# axis = 2를 기준으로 평균
predictions_array = np.concatenate(predictions_list, axis = 2)
predictions_mean = predictions_array.mean(axis = 2)

# 평균 값이 0.5보다 클 경우 1 작으면 0
# predictions_mean = (predictions_mean > 0.5) * 1
predictions_mean

array([[0.99842417, 0.01340955, 0.99698669, ..., 0.98389763, 0.05129382,
        0.99686313],
       [0.01622772, 0.97145128, 0.74581873, ..., 0.0094005 , 0.03873951,
        0.01397544],
       [0.05934459, 0.02973176, 0.87512255, ..., 0.02692127, 0.06540667,
        0.9749732 ],
       ...,
       [0.96252429, 0.0143478 , 0.05164703, ..., 0.0369614 , 0.03178167,
        0.98216665],
       [0.05198433, 0.01811621, 0.98821974, ..., 0.0620117 , 0.13981763,
        0.99647141],
       [0.99355996, 0.91200703, 0.07288954, ..., 0.97407192, 0.99924099,
        0.99101263]])

## 8. 제출파일 생성

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.iloc[:,1:] = predictions_mean
sample_submission.to_csv("EfficientNet_0.926.csv", index = False)
sample_submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,0.998424,0.013410,0.996987,0.942028,0.992872,0.999403,0.017614,0.945488,0.998873,0.637929,0.017881,0.029777,0.022418,0.042058,0.048763,0.095549,0.055655,0.065010,0.012008,0.657257,0.994691,0.926540,0.017496,0.983898,0.051294,0.996863
1,50001,0.016228,0.971451,0.745819,0.062009,0.999047,0.019482,0.993740,0.046878,0.976278,0.547279,0.985005,0.980480,0.014663,0.012684,0.936382,0.993138,0.324497,0.603298,0.054533,0.030471,0.996134,0.993582,0.020829,0.009400,0.038740,0.013975
2,50002,0.059345,0.029732,0.875123,0.997460,0.978672,0.022515,0.991150,0.356594,0.999753,0.327631,0.921705,0.893846,0.224765,0.565062,0.015607,0.998544,0.021534,0.974193,0.999724,0.047407,0.995504,0.182060,0.996152,0.026921,0.065407,0.974973
3,50003,0.991719,0.970988,0.017428,0.120970,0.002542,0.935888,0.994521,0.135545,0.975706,0.069510,0.997831,0.964497,0.987209,0.014662,0.007123,0.990588,0.023332,0.948096,0.025124,0.999339,0.005154,0.039242,0.035161,0.994571,0.109704,0.970242
4,50004,0.020268,0.006498,0.975062,0.013067,0.997252,0.991559,0.017233,0.011030,0.025967,0.023196,0.007787,0.966103,0.998403,0.013573,0.987371,0.108054,0.994401,0.424204,0.996948,0.999335,0.629784,0.201701,0.998897,0.008450,0.189819,0.059704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0.035252,0.933776,0.995015,0.278911,0.012578,0.044641,0.990105,0.997664,0.059357,0.226412,0.745159,0.644660,0.033969,0.544838,0.962749,0.009920,0.047023,0.899548,0.031221,0.972405,0.458758,0.621913,0.995556,0.043435,0.991228,0.145159
4996,54996,0.987062,0.819498,0.996045,0.055221,0.195513,0.155926,0.261240,0.228457,0.877037,0.992175,0.031062,0.789515,0.072910,0.056536,0.982915,0.978822,0.295691,0.057267,0.723913,0.094607,0.046266,0.192034,0.041330,0.972057,0.134651,0.994649
4997,54997,0.962524,0.014348,0.051647,0.994151,0.023805,0.989746,0.006073,0.019636,0.326801,0.379597,0.985785,0.611621,0.944593,0.032773,0.998933,0.998070,0.008864,0.167909,0.011464,0.993763,0.981670,0.999448,0.978114,0.036961,0.031782,0.982167
4998,54998,0.051984,0.018116,0.988220,0.032516,0.010978,0.201375,0.992291,0.214815,0.704719,0.975172,0.062060,0.608052,0.038642,0.046520,0.034405,0.025420,0.998280,0.998160,0.023423,0.988335,0.215334,0.048262,0.999095,0.062012,0.139818,0.996471
